In [1]:
import pynetbox
import easysnmp
import re
import os
import ipaddress
import paramiko

In [10]:
key = os.environ['NETBOX_TOKEN']
nb = pynetbox.api(
    'https://netbox.rc.umass.edu/',
    token=key
)
ssh_client = paramiko.SSHClient()
ssh_user = os.environ['SSH_USER']
router_prefix = "2001:48e8:6401::/64"

In [11]:
#copied from wido/mac2eui64.py
def mac2eui64(mac, prefix=None):
    '''
    Convert a MAC address to a EUI64 address
    or, with prefix provided, a full IPv6 address
    '''
    # http://tools.ietf.org/html/rfc4291#section-2.5.1
    eui64 = re.sub(r'[.:-]', '', mac).lower()
    eui64 = eui64[0:6] + 'fffe' + eui64[6:]
    eui64 = hex(int(eui64[0:2], 16) ^ 2)[2:].zfill(2) + eui64[2:]

    if prefix is None:
        return ':'.join(re.findall(r'.{4}', eui64))
    else:
        try:
            net = ipaddress.ip_network(prefix, strict=False)
            euil = int('0x{0}'.format(eui64), 16)
            return str(net[euil])
        except:  # pylint: disable=bare-except
            return

In [12]:
def getInfo(ip, nb):
    mac_3033 = ''
    mac_3031 = ''
    mac_59 = ''
    nic_3031 = ''   
    nic_3033 = ''
    nic_59 = ''
    dev_name = ''
    dev_tag = ''
    dev_model = ''
    ip_query = nb.ipam.ip_addresses.filter(ip)
    if (len(ip_query) > 1):
        raise Exception("More than one IP with this address found")
    device = [i.assigned_object.device for i in ip_query]
    device = nb.dcim.devices.get(device[0].id)
    dev_name = device.name
    dev_tag = device.serial
    dev_model = device.device_type
    interfaces = nb.dcim.interfaces.filter(device=device)
    for interface in interfaces:
        ip_interface = nb.ipam.ip_addresses.filter(interface_id=interface.id)
        for ip in ip_interface:
            subnet = int(ip.address.split('.')[2])
            if subnet in [172, 173]:
                mac_3031 = interface.mac_address
                nic_3031 = interface.name
            elif subnet in [174, 175]:
                mac_3033 = interface.mac_address
                nic_3033 = interface.name
            elif subnet in [44, 45]:
                mac_59 = interface.mac_address
                nic_59 = interface.name
    ipv6_59 =  mac2eui64(mac_59, router_prefix)
    print(mac_3031, nic_3031, mac_3033, nic_3033, mac_59, nic_59, ipv6_59, dev_name, dev_tag, dev_model)

In [13]:
def getInternalPorts(rack, client, user):
    p_mac = re.compile(r'(?:[0-9a-fA-F]:?){12}')
    p_port = re.compile(r'(?:xe|ge)-\d+/\d+/\d+')
    p_vlan = re.compile(r'(?:mghpcc-atlas-mgmt-1|mghpcc-net2-mgmt|net2-openshift-prov-1)')
    vlan_dict = {'mghpcc-atlas-mgmt-1': 58, 'mghpcc-net2-mgmt': 3031, 'net2-openshift-prov-1': 3033}
    if rack=='13':
    
        # Automatically add the host key
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
        try:
            client.connect(hostname='172.20.170.10', username=user)

            stdin, stdout, stderr = client.exec_command('show ethernet-switching table brief')
            
            output = stdout.read().decode()
            macs = re.findall(p_mac, output)
            ports = re.findall(p_port, output)
            vlans = re.findall(p_vlan, output)
            return dict(zip(macs, ports)), dict(zip(macs, [vlan_dict[i] for i in vlans]))
            
        finally:
            # Close the SSH client
            client.close()

In [18]:
def getPublicPorts(rack, client, user):
    p_mac = re.compile(r'(?:[0-9a-fA-F]:?){12}')
    p_port = re.compile(r'ethernet\d+/\d+/\d+')
    p_vlan = re.compile(r'\b(?:58|59|3031|3033|3034)\b')
    vlan_dict = {'mghpcc-atlas-mgmt-1': 58, 'mghpcc-net2-mgmt': 3031, 'net2-openshift-prov-1': 3033}
    if rack=='13':
    
        # Automatically add the host key
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
        try:
            client.connect(hostname='172.20.170.11', username=user)

            stdin, stdout, stderr = client.exec_command('show mac address-table')
            
            output = stdout.read().decode()
            macs = re.findall(p_mac, output)
            ports = re.findall(p_port, output)
            vlans = re.findall(p_vlan, output)
            return dict(zip(macs, ports)), dict(zip(macs, vlans))
            
        finally:
            # Close the SSH client
            client.close()

In [15]:
def getInterfaces(ip, model):
    if 'abc-p' in model:
        nic_names = ['BMC']
        mac = easysnmp.snmp_get('1.3.6.1.2.1.2.2.1.6.2', hostname=ip, community='public', version=2).value.encode("latin-1").hex()
        mac = ':'.join(str(mac)[i:i+2] for i in range(0,len(str(mac)),2))
        nic_phys = [mac]
        nic_names.append(easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.30.1.1', hostname=ip, community='public', version=2).value)
        nic_names.append(easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.30.1.2', hostname=ip, community='public', version=2).value)
        nic_names.append(easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.30.1.3', hostname=ip, community='public', version=2).value)
        nic_names.append(easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.30.1.4', hostname=ip, community='public', version=2).value)
        mac = easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.15.1.1', hostname=ip, community='public', version=2).value.encode("latin-1").hex()
        nic_phys.append(':'.join(str(mac)[i:i+2] for i in range(0,len(str(mac)),2)))     
        mac = easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.15.1.2', hostname=ip, community='public', version=2).value.encode("latin-1").hex()
        nic_phys.append(':'.join(str(mac)[i:i+2] for i in range(0,len(str(mac)),2)))     
        mac = easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.15.1.3', hostname=ip, community='public', version=2).value.encode("latin-1").hex()
        nic_phys.append(':'.join(str(mac)[i:i+2] for i in range(0,len(str(mac)),2)))     
        mac = easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.15.1.4', hostname=ip, community='public', version=2).value.encode("latin-1").hex()
        nic_phys.append(':'.join(str(mac)[i:i+2] for i in range(0,len(str(mac)),2)))
        return dict(zip(nic_phys, nic_names))
    elif 'abc-q' in model:
        nic_names = ['BMC']
        mac = easysnmp.snmp_get('1.3.6.1.2.1.2.2.1.6.2', hostname=ip, community='public', version=2).value.encode("latin-1").hex()
        mac = ':'.join(str(mac)[i:i+2] for i in range(0,len(str(mac)),2))
        nic_phys = [mac]
        nic_names.append(easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.30.1.1', hostname=ip, community='public', version=2).value)
        nic_names.append(easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.30.1.2', hostname=ip, community='public', version=2).value)
        nic_names.append(easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.30.1.3', hostname=ip, community='public', version=2).value)
        nic_names.append(easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.30.1.4', hostname=ip, community='public', version=2).value)
        mac = easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.15.1.1', hostname=ip, community='public', version=2).value.encode("latin-1").hex()
        nic_phys.append(':'.join(str(mac)[i:i+2] for i in range(0,len(str(mac)),2)))     
        mac = easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.15.1.2', hostname=ip, community='public', version=2).value.encode("latin-1").hex()
        nic_phys.append(':'.join(str(mac)[i:i+2] for i in range(0,len(str(mac)),2)))     
        mac = easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.15.1.3', hostname=ip, community='public', version=2).value.encode("latin-1").hex()
        nic_phys.append(':'.join(str(mac)[i:i+2] for i in range(0,len(str(mac)),2)))     
        mac = easysnmp.snmp_get('1.3.6.1.4.1.674.10892.5.4.1100.90.1.15.1.4', hostname=ip, community='public', version=2).value.encode("latin-1").hex()
        nic_phys.append(':'.join(str(mac)[i:i+2] for i in range(0,len(str(mac)),2)))
        return dict(zip(nic_phys, nic_names))        

In [16]:
print(getInfo('172.20.173.106', nb))

B0:7B:25:F0:BB:F0 BMC B0:7B:25:DE:4E:60 NIC.Embedded.1-1-1 5C:6F:69:7C:79:C0 NIC.Integrated.1-1-1 2001:48e8:6401:0:5e6f:69ff:fe7c:79c0 NET2-node106 DW9RKM3 PowerEdge R6525
None


In [19]:
internal_ports, internal_vlans = getInternalPorts('13', ssh_client, ssh_user)
public_ports, public_vlans = getPublicPorts('13', ssh_client, ssh_user)


b0:7b:25:f0:bb:f0 BMC ge-0/0/31 3031
b0:7b:25:de:4e:60 NIC.Embedded.1-1-1 ge-0/0/9 3033
5c:6f:69:7c:79:c1 NIC.Integrated.1-2-1
5c:6f:69:7c:79:c0 NIC.Integrated.1-1-1 ethernet1/1/37 59
b0:7b:25:de:4e:61 NIC.Embedded.2-1-1
